In [1]:
from marthas_dashboard.api import *
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [19]:
api = API()

# THIS IS TOO MUCH DATA FOR OUR API, WE NEED TO DISCUSS THIS, 2 DAYS BREAKS IT???
# data = api.building_values_in_range('4', '2016-08-18', '2017-08-19')
data = api.building_values_in_range('4', '2017-08-18 12:00', '2017-08-18 20:00')

data.head()
data.shape

HERE http://energycomps.its.carleton.edu/api/index.php/values/building/4/2017-08-18 12:00/2017-08-18 20:00


(6107, 9)

In [20]:
data.head()

,factor,id,name,pointid,pointname,pointtimestamp,pointvalue,returntype,units
0,2,47,floatCFM2,526,BI2CFM,2017-08-18 12:15:00,18084.83,float,CFM
1,2,47,floatCFM2,526,BI2CFM,2017-08-18 12:30:00,18398.29,float,CFM
2,2,47,floatCFM2,526,BI2CFM,2017-08-18 12:45:00,18330.83,float,CFM
3,2,47,floatCFM2,526,BI2CFM,2017-08-18 13:00:00,18158.46,float,CFM
4,2,47,floatCFM2,526,BI2CFM,2017-08-18 13:15:00,18226.55,float,CFM


In [21]:
basket = (data
          .groupby(['pointtimestamp', 'pointname'])['pointvalue']
          .sum().unstack().reset_index().fillna(0)
          .set_index('pointtimestamp'))
basket.head()

pointname,BI.LAB.HW.TEMP,BI1CCT,BI1CFM,BI1DSC,BI1DSP,BI1HMC,BI1HRT,BI1RRC,BI1RRT,BI1SAC,...,HU.RB02.RM,HU.RB02.RMT,HU.RB02.SCFM,HU.RB02.TEXH,HU.RB02.TEXHO,HU.RB02A.RHV,HU.RB02A.RMT,HU.RB03.RM,HU.RB03.RMT,HU.RB03.SCFM
pointtimestamp,,,,,,,,,,,,,,,,,,,,,
2017-08-18 12:15:00,148.44,52.99,8181.85,1.5,1.51,35.0,67.87,32.0,82.34,55.0,...,70.21,70.21,467.81,462.93,0.0,3.0,69.78,72.49,72.49,204.37
2017-08-18 12:30:00,146.92,50.54,8123.26,1.5,1.50,35.0,67.62,32.0,82.31,55.0,...,69.42,69.42,457.27,461.94,0.0,3.0,69.06,71.87,71.87,201.78
2017-08-18 12:45:00,151.63,50.43,8025.61,1.5,1.49,35.0,67.80,32.0,82.34,55.0,...,70.10,70.10,463.13,465.82,0.0,3.0,69.53,71.40,71.40,199.34
2017-08-18 13:00:00,149.37,50.67,8108.32,1.5,1.51,35.0,67.69,32.0,82.31,55.0,...,70.45,70.45,450.85,463.08,0.0,3.0,69.84,71.50,71.50,203.00
2017-08-18 13:15:00,149.37,50.67,8108.32,1.5,1.51,35.0,67.69,32.0,82.31,55.0,...,69.53,69.53,455.35,466.28,0.0,3.0,69.09,72.03,72.03,203.84


In [22]:
for column in basket:
    if "HU" in column:
        basket.drop([column], axis = 1, inplace = True)
basket.head()

pointname,BI.LAB.HW.TEMP,BI1CCT,BI1CFM,BI1DSC,BI1DSP,BI1HMC,BI1HRT,BI1RRC,BI1RRT,BI1SAC,...,BIE20C,BIE20P,BIE21C,BIE21P,BIE23P,BIE25C,BIE25P,BIHRRT,BIHRST,BIOAT27
pointtimestamp,,,,,,,,,,,,,,,,,,,,,
2017-08-18 12:15:00,148.44,52.99,8181.85,1.5,1.51,35.0,67.87,32.0,82.34,55.0,...,1.2,1.20,1.2,1.21,1.22,2.0,1.57,81.87,81.44,65.47
2017-08-18 12:30:00,146.92,50.54,8123.26,1.5,1.50,35.0,67.62,32.0,82.31,55.0,...,1.2,1.18,1.2,1.19,1.20,2.0,1.55,81.91,81.44,65.38
2017-08-18 12:45:00,151.63,50.43,8025.61,1.5,1.49,35.0,67.80,32.0,82.34,55.0,...,1.2,1.21,1.2,1.22,1.20,2.0,1.61,81.91,81.44,65.27
2017-08-18 13:00:00,149.37,50.67,8108.32,1.5,1.51,35.0,67.69,32.0,82.31,55.0,...,1.2,1.21,1.2,1.22,1.21,2.0,1.61,81.87,81.41,65.12
2017-08-18 13:15:00,149.37,50.67,8108.32,1.5,1.51,35.0,67.69,32.0,82.31,55.0,...,1.2,1.21,1.2,1.16,1.19,2.0,1.58,81.87,81.41,65.12


In [7]:
# data.drop(['factor'], axis = 1, inplace = True)
# data.drop(['id'], axis = 1, inplace = True)
# data.drop(['name'], axis = 1, inplace = True)
# data.drop(['pointid'], axis = 1, inplace = True)
# data.drop(['returntype'], axis = 1, inplace = True)
# data.drop(['units'], axis = 1, inplace = True)
# data.head()
# data.shape

In [8]:
# data = data.pivot(index='pointtimestamp', columns='pointname', values='pointvalue')
# data.head()
# for column in data:
#     if "H" in column:
#         data.drop([column], axis = 1, inplace = True)
# data.shape

In [23]:
# Bin the data's point values into 100 different bins (all of equal size)
for column in basket:
    basket[column] = pd.cut(basket[column], 100)
basket.head()

pointname,BI.LAB.HW.TEMP,BI1CCT,BI1CFM,BI1DSC,BI1DSP,BI1HMC,BI1HRT,BI1RRC,BI1RRT,BI1SAC,...,BIE20C,BIE20P,BIE21C,BIE21P,BIE23P,BIE25C,BIE25P,BIHRRT,BIHRST,BIOAT27
pointtimestamp,,,,,,,,,,,,,,,,,,,,,
2017-08-18 12:15:00,"(148.36, 148.556]","(52.969, 53.017]","(8180.172, 8184.873]","(1.49997, 1.5]","(1.5098, 1.51]","(34.9993, 35.0]","(67.846, 67.914]","(31.9994, 32.0]","(82.34, 82.345]","(54.999, 55.0]",...,"(1.19998, 1.2]","(1.199, 1.2]","(1.19998, 1.2]","(1.209, 1.21]","(1.2197, 1.22]","(1.99996, 2.0]","(1.569, 1.57]","(81.869, 81.87]","(81.439, 81.44]","(65.465, 65.531]"
2017-08-18 12:30:00,"(146.792, 146.988]","(50.526, 50.574]","(8119.056, 8123.757]","(1.49997, 1.5]","(1.4998, 1.5]","(34.9993, 35.0]","(67.575, 67.643]","(31.9994, 32.0]","(82.305, 82.31]","(54.999, 55.0]",...,"(1.19998, 1.2]","(1.179, 1.18]","(1.19998, 1.2]","(1.189, 1.19]","(1.1999, 1.2002]","(1.99996, 2.0]","(1.55, 1.551]","(81.91, 81.911]","(81.439, 81.44]","(65.334, 65.399]"
2017-08-18 12:45:00,"(151.494, 151.69]","(50.425, 50.478]","(8025.032, 8029.733]","(1.49997, 1.5]","(1.49, 1.4902]","(34.9993, 35.0]","(67.778, 67.846]","(31.9994, 32.0]","(82.34, 82.345]","(54.999, 55.0]",...,"(1.19998, 1.2]","(1.209, 1.21]","(1.19998, 1.2]","(1.219, 1.22]","(1.1999, 1.2002]","(1.99996, 2.0]","(1.609, 1.61]","(81.91, 81.911]","(81.439, 81.44]","(65.268, 65.334]"
2017-08-18 13:00:00,"(149.339, 149.535]","(50.67, 50.717]","(8104.952, 8109.654]","(1.49997, 1.5]","(1.5098, 1.51]","(34.9993, 35.0]","(67.643, 67.711]","(31.9994, 32.0]","(82.305, 82.31]","(54.999, 55.0]",...,"(1.19998, 1.2]","(1.209, 1.21]","(1.19998, 1.2]","(1.219, 1.22]","(1.2098, 1.2101]","(1.99996, 2.0]","(1.609, 1.61]","(81.869, 81.87]","(81.41, 81.411]","(65.063, 65.136]"
2017-08-18 13:15:00,"(149.339, 149.535]","(50.67, 50.717]","(8104.952, 8109.654]","(1.49997, 1.5]","(1.5098, 1.51]","(34.9993, 35.0]","(67.643, 67.711]","(31.9994, 32.0]","(82.305, 82.31]","(54.999, 55.0]",...,"(1.19998, 1.2]","(1.209, 1.21]","(1.19998, 1.2]","(1.159, 1.16]","(1.19, 1.1903]","(1.99996, 2.0]","(1.579, 1.58]","(81.869, 81.87]","(81.41, 81.411]","(65.063, 65.136]"


In [24]:
result = pd.get_dummies(basket)
result.head()

,"BI.LAB.HW.TEMP_(139.72, 139.936]","BI.LAB.HW.TEMP_(139.936, 140.132]","BI.LAB.HW.TEMP_(140.132, 140.328]","BI.LAB.HW.TEMP_(140.328, 140.524]","BI.LAB.HW.TEMP_(140.524, 140.72]","BI.LAB.HW.TEMP_(140.72, 140.915]","BI.LAB.HW.TEMP_(140.915, 141.111]","BI.LAB.HW.TEMP_(141.111, 141.307]","BI.LAB.HW.TEMP_(141.307, 141.503]","BI.LAB.HW.TEMP_(141.503, 141.699]",...,"BIOAT27_(71.001, 71.067]","BIOAT27_(71.067, 71.133]","BIOAT27_(71.133, 71.199]","BIOAT27_(71.199, 71.265]","BIOAT27_(71.265, 71.33]","BIOAT27_(71.33, 71.396]","BIOAT27_(71.396, 71.462]","BIOAT27_(71.462, 71.528]","BIOAT27_(71.528, 71.594]","BIOAT27_(71.594, 71.66]"
pointtimestamp,,,,,,,,,,,,,,,,,,,,,
2017-08-18 12:15:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-08-18 12:30:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-08-18 12:45:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-08-18 13:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017-08-18 13:15:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
result = result.to_sparse(fill_value=0)

In [30]:
# Frequent items
frq_itemsets = apriori(result, min_support=0.8, use_colnames=True)
#frq_itemsets = frequent_itemsets(result, min_support=0.2)
frq_itemsets

,support,itemsets
0,1.0,"[BI1DSC_(1.49997, 1.5]]"
1,1.0,"[BI1HMC_(34.9993, 35.0]]"
2,1.0,"[BI1RRC_(31.9994, 32.0]]"
3,1.0,"[BI1SAC_(54.999, 55.0]]"
4,1.0,"[BI2DSC_(1.49997, 1.5]]"


In [ ]:
# Generate association rules
rules = association_rules(frq_itemsets, metric="lift")
rules